In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
import numpy as np
import pandas as pd

In [47]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [48]:
df = pd.read_csv('/content/drive/MyDrive/day28-Column-transformer/covid_toy.csv')
df.shape

(100, 6)

In [49]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [50]:
X = df.drop(columns=['has_covid'])
y = df['has_covid']

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [52]:
X_train

,age,gender,fever,cough,city
28,16,Male,104.0,Mild,Kolkata
85,16,Female,103.0,Mild,Bangalore
69,73,Female,103.0,Mild,Delhi
5,84,Female,NaN,Mild,Bangalore
2,42,Male,101.0,Mild,Delhi
...,...,...,...,...,...
40,49,Female,102.0,Mild,Delhi
64,42,Male,104.0,Mild,Mumbai
84,69,Female,98.0,Strong,Mumbai
92,82,Female,102.0,Strong,Kolkata


## 1. Aam Zindagi (Without Column transformer)

In [53]:
#adding simple imputer to fever column
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [54]:
df.columns

Index(['age', 'gender', 'fever', 'cough', 'city', 'has_covid'], dtype='object')

In [55]:
X_train['cough'].value_counts()

,count
cough,
Mild,47
Strong,33


In [56]:
# Ordinal encoding -> cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [57]:
# One-Hot Encoding -> gender, city columns
ohe = OneHotEncoder(drop='first', sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [58]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [59]:
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)

# also the test data
X_test_transformed = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)



In [60]:
print(X_train_transformed.shape)
print(X_test_transformed.shape)

(80, 7)
(20, 7)


In [61]:
pd.DataFrame(X_train_transformed, columns=['age', 'fever', 'gc01', 'gc02', 'gc03', 'gc04', 'cough'])

,age,fever,gc01,gc02,gc03,gc04,cough
0,16.0,104.000000,1.0,0.0,1.0,0.0,0.0
1,16.0,103.000000,0.0,0.0,0.0,0.0,0.0
2,73.0,103.000000,0.0,1.0,0.0,0.0,0.0
3,84.0,100.929577,0.0,0.0,0.0,0.0,0.0
4,42.0,101.000000,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
75,49.0,102.000000,0.0,1.0,0.0,0.0,0.0
76,42.0,104.000000,1.0,0.0,0.0,1.0,0.0
77,69.0,98.000000,0.0,0.0,0.0,1.0,1.0
78,82.0,102.000000,0.0,0.0,1.0,0.0,1.0


Mentos Zindagi (With Column transformer)

In [62]:
from sklearn.compose import ColumnTransformer

In [63]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [66]:
X_train_encoded = transformer.fit_transform(X_train)

X_test_encoded = transformer.fit_transform(X_test)

print(X_train_encoded.shape)
print(X_test_encoded.shape)

(80, 7)
(20, 7)
